In [1]:
# Function file for data analysis and all ML classifiers.

In [2]:
# 2 classes (imbalanced) and 8 csv files
# Data function file

import numpy as np
import pandas as pd
import scipy.io as sio
import collections
from sklearn import metrics
from collections import Counter
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, log_loss, f1_score
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from imblearn.over_sampling import SMOTE
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef

In [3]:
# Function for Box plot

def drawBarPlot(sns,plt,predictors,target,new_df):
    i =0
    predictors[i]
    len(predictors)
    plt.figure(figsize=(30,30))
    k=0
    for i in range(1,len(predictors)+1):
        plt.subplot(5, 3, i)
        ax = sns.boxplot(x=target, y=predictors[k], data=new_df)
        ax.set_ylabel(predictors[k], fontsize = 16)
        k = k + 1
        if k == len(predictors):
            break

    plt.show()

In [4]:
# Function for Heatmap

def drawHeatMap(df_heatmap):
    import plotly
    import plotly.graph_objs as go
    import plotly.figure_factory as ff
    from plotly.offline import init_notebook_mode
    init_notebook_mode(connected=True) # to show plots in notebook

    # online plotly
    from plotly.plotly import plot, iplot
    plotly.tools.set_credentials_file(username='rkj08105', api_key='TU0mevAzP1XtMWeQ5xig')
    corr = df_heatmap.corr()
    trace = go.Heatmap(z=corr.values.tolist(), x=corr.columns, y=corr.columns)
    data=[trace]
    layout = go.Layout(
        title='Heatmap of pairwise correlation of the columns',
        autosize=True,
        width=750,
        height=750,
        yaxis=go.layout.YAxis(automargin=True),
        xaxis=dict(tickangle=40),
        margin=go.layout.Margin(l=0, r=200, b=200, t=80)
    )
    return iplot,go,layout,data

In [5]:
# Function for Normalization

def getNormalization(df):
    #df_norm=df.drop(['sample'], axis=1)  # Needed column
    df_norm=df
    header = list(df_norm)  
    i=0
    for i in range(len(header)-1):
        df_norm[header[i]] = (df_norm[header[i]] - df_norm[header[i]] .mean())/df_norm[header[i]] .std()
    return df_norm;

In [6]:
# Function for Histogram plot

def drawHistogramPlot(sns,plt,df_norm,predictors):
    i=0
    predictors[i]
    plt.figure(figsize=(30,20))
    k=0
    for i in range(1,len(predictors)+1):
        plt.subplot(4, 4, i)
        sns.distplot(df_norm[predictors[k]])
        k = k + 1
        if k == len(predictors):
            break

    plt.show()

In [7]:
# Function to get different dataframe

def getDifferentDataFrame(df,sample1):
    
    for index, row in df.iterrows():
        #print(index, row['sample'])
        sample=str(row['sample'])
        sample=sample[5:-4]
        q = sample1 
#         q =q[5:]
        findpos = sample.find(q)
        #print(findpos)
        if findpos != -1:
            df.at[index, 'Class'] = 0
        else:
            df.at[index, 'Class'] = 1
    return df

In [8]:
# Function to split data # Stratified split

def getTrainTestData(data, Number_of_Samples, random_state1, random_state2): 
    class2 = data[data.Class == 1].sample(n=Number_of_Samples,random_state=random_state1)
    data = data[data.Class !=1]
    data = pd.concat([data,class2], axis=0)
    data = shuffle(data,random_state=random_state1).reset_index()
    X = data.iloc[:,2:-1]
    Y = data.iloc[:,-1]
    
    # Balancing Data

    print('Original dataset shape {}'.format(Counter(Y)))
    smt = SMOTE(random_state=random_state1)
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, stratify=Y, random_state=random_state2)
    X_balanced, Y_balanced = smt.fit_sample(x_train, y_train)
    X_balanced = pd.DataFrame(X_balanced,columns=X.columns)
    x_train,y_train = X_balanced,Y_balanced

    print('New dataset shape {}'.format(Counter(Y_balanced)))
    return x_train, y_train, x_test, y_test


In [9]:
# Function for different ML Classifiers 

def Report(x_train, y_train, x_test, y_test):
    print("################# XGBoost ########################")
    classifier = xgb.XGBRegressor(booster='dart',objective ='multi:softmax', num_class=3,learning_rate = 0.1,max_depth = 100, alpha = 10, n_estimators = 1000)
    classifier.fit(x_train, y_train)
    y_pred = classifier.predict(x_test)
    print("G-Mean:",geometric_mean_score(y_test, y_pred))
    print("Matthews correlation coefficient:", matthews_corrcoef(y_test, y_pred))
    print('Algorithm:{} , Accuracy:{}'.format("XGBoost",accuracy_score(y_test, y_pred)))
    print("Confusion Matrix:\n",metrics.confusion_matrix(y_test,y_pred))
    print("Classification Report:\n",metrics.classification_report(y_test,y_pred))
    
    # 10-fold cross validation accuracy plot
    accuracies = cross_val_score(estimator = classifier, X = x_train, y = y_train, cv = 10)
    x_range = [x for x in range(1,11)]
    ax = sns.lineplot(x=x_range,y=accuracies,markers=True,dashes=False)
    ax.set_xticks(x_range)
    ax.set_title('XGBoost 10 fold accuracy')
    ax.set_ylabel('Acuuracy')
    ax.set_xlabel('10 Fold')
    plt.show()
    
    ######################################################################################
    
    print("############### Random Forest #####################")
    classifier = RandomForestClassifier(n_estimators=501,random_state=0)
    classifier.fit(x_train, y_train)
    y_pred = classifier.predict(x_test)
    print("G-Mean:",geometric_mean_score(y_test, y_pred))
    print("Matthews correlation coefficient:", matthews_corrcoef(y_test, y_pred))
    print('Algorithm:{} , Accuracy:{}'.format("Random Forest",accuracy_score(y_test, y_pred)))
    print("Confusion Matrix:\n",metrics.confusion_matrix(y_test,y_pred))
    print("Classification Report:\n",metrics.classification_report(y_test,y_pred))
    
    # 10-fold cross validation accuracy plot
    accuracies = cross_val_score(estimator = classifier, X = x_train, y = y_train, cv = 10)
    x_range = [x for x in range(1,11)]
    ax = sns.lineplot(x=x_range,y=accuracies,markers=True,dashes=False)
    ax.set_xticks(x_range)
    ax.set_title('Random Forest 10 fold accuracy')
    ax.set_ylabel('Acuuracy')
    ax.set_xlabel('10 Fold')
    plt.show()
    
    # ROC graph plot and AUC score
    y_pred_proba = classifier.predict_proba(x_test)[:,1]
    fpr, tpr, threshold = metrics.roc_curve(y_test,  y_pred_proba)
    roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)
    plt.plot(fpr,tpr,label="RF, auc="+str(roc_auc))
    plt.title('ROC curve of Random Forest')
    plt.legend(loc= 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
    
    ########################################################################################
    
    print("############### Decision Tree #######################")
    classifier = DecisionTreeClassifier(random_state=0)
    classifier.fit(x_train, y_train)
    y_pred = classifier.predict(x_test)
    print("G-Mean:",geometric_mean_score(y_test, y_pred))
    print("Matthews correlation coefficient:", matthews_corrcoef(y_test, y_pred))
    print('Algorithm:{} , Accuracy:{}'.format("Decision Tree",accuracy_score(y_test, y_pred)))
    print("Confusion Matrix:\n",metrics.confusion_matrix(y_test,y_pred))
    print("Classification Report:\n",metrics.classification_report(y_test,y_pred))
    
    # 10-fold cross validation accuracy plot
    accuracies = cross_val_score(estimator = classifier, X = x_train, y = y_train, cv = 10)
    x_range = [x for x in range(1,11)]
    ax = sns.lineplot(x=x_range,y=accuracies,markers=True,dashes=False)
    ax.set_xticks(x_range)
    ax.set_title('Decision Tree 10 fold accuracy')
    ax.set_ylabel('Acuuracy')
    ax.set_xlabel('10 Fold')
    plt.show()
    
    # ROC graph plot and AUC score
    y_pred_proba = classifier.predict_proba(x_test)[:,1]
    fpr, tpr, threshold = metrics.roc_curve(y_test,  y_pred_proba)
    roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)
    plt.plot(fpr,tpr,label="DT, auc="+str(roc_auc))
    plt.title('ROC curve of Decision Tree')
    plt.legend(loc= 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

#     ##########################################################################################
    
    print("############### Logistic Regression ###################")
    classifier = LogisticRegression(solver="saga",multi_class="auto",max_iter=10000,random_state = 0,C=1,fit_intercept=True)
    classifier.fit(x_train, y_train)
    y_pred = classifier.predict(x_test)
    print("G-Mean:",geometric_mean_score(y_test, y_pred))
    print("Matthews correlation coefficient:", matthews_corrcoef(y_test, y_pred))
    print('Algorithm:{} , Accuracy:{}'.format("Logistic Regression",accuracy_score(y_test, y_pred)))
    print("Confusion Matrix:\n",metrics.confusion_matrix(y_test,y_pred))
    print("Classification Report:\n",metrics.classification_report(y_test,y_pred))
    
    # 10-fold cross validation accuracy plot
    accuracies = cross_val_score(estimator = classifier, X = x_train, y = y_train, cv = 10)
    x_range = [x for x in range(1,11)]
    ax = sns.lineplot(x=x_range,y=accuracies,markers=True,dashes=False)
    ax.set_xticks(x_range)
    ax.set_title('Logistic Regression 10 fold accuracy')
    ax.set_ylabel('Acuuracy')
    ax.set_xlabel('10 Fold')
    plt.show()
    
    # ROC graph plot and AUC score
    y_pred_proba = classifier.predict_proba(x_test)[:,1]
    fpr, tpr, threshold = metrics.roc_curve(y_test,  y_pred_proba)
    roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)
    plt.plot(fpr,tpr,label="LR, auc="+str(roc_auc))
    plt.title('ROC curve of Logistic Regression')
    plt.legend(loc= 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
    
    
#     ##############################################################################################
    
    print("###################### SVM #############################")
    classifier = SVC(kernel='poly',gamma="scale",degree=5)
    classifier.fit(x_train, y_train)
    y_pred = classifier.predict(x_test)
    print("G-Mean:",geometric_mean_score(y_test, y_pred))
    print("Matthews correlation coefficient:", matthews_corrcoef(y_test, y_pred))
    print('Algorithm:{} , Accuracy:{}'.format("SVM",accuracy_score(y_test, y_pred)))
    print("Confusion Matrix:\n",metrics.confusion_matrix(y_test,y_pred))
    print("Classification Report:\n",metrics.classification_report(y_test,y_pred))
    
    # 10-fold cross validation accuracy plot
    accuracies = cross_val_score(estimator = classifier, X = x_train, y = y_train, cv = 10)
    x_range = [x for x in range(1,11)]
    ax = sns.lineplot(x=x_range,y=accuracies,markers=True,dashes=False)
    ax.set_xticks(x_range)
    ax.set_title('SVM 10 fold accuracy')
    ax.set_ylabel('Acuuracy')
    ax.set_xlabel('10 Fold')
    plt.show()
    
#     #################################################################################
    
    print("###################### KNN ###############################")
    classifier = KNeighborsClassifier()
    classifier.fit(x_train, y_train)
    y_pred = classifier.predict(x_test)
    print("G-Mean:",geometric_mean_score(y_test, y_pred))
    print("Matthews correlation coefficient:", matthews_corrcoef(y_test, y_pred))
    print('Algorithm:{} , Accuracy:{}'.format("KNN",accuracy_score(y_test, y_pred)))
    print("Confusion Matrix:\n",metrics.confusion_matrix(y_test,y_pred))
    print("Classification Report:\n",metrics.classification_report(y_test,y_pred))
    
    # 10-fold cross validation accuracy plot
    accuracies = cross_val_score(estimator = classifier, X = x_train, y = y_train, cv = 10)
    x_range = [x for x in range(1,11)]
    ax = sns.lineplot(x=x_range,y=accuracies,markers=True,dashes=False)
    ax.set_xticks(x_range)
    ax.set_title('KNN 10 fold accuracy')
    ax.set_ylabel('Acuuracy')
    ax.set_xlabel('10 Fold')
    plt.show()
    
    # ROC graph plot and AUC score
    y_pred_proba = classifier.predict_proba(x_test)[:,1]
    fpr, tpr, threshold = metrics.roc_curve(y_test,  y_pred_proba)
    roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)
    plt.plot(fpr,tpr,label="KNN, auc="+str(roc_auc))
    plt.title('ROC curve of KNN')
    plt.legend(loc= 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
    
#     #################################################################################
    
    print("###################### Linear SVC #########################")
    classifier = LinearSVC(max_iter=10000)
    classifier.fit(x_train, y_train)
    y_pred = classifier.predict(x_test)
    print("G-Mean:",geometric_mean_score(y_test, y_pred))
    print("Matthews correlation coefficient:", matthews_corrcoef(y_test, y_pred))
    print('Algorithm:{} , Accuracy:{}'.format("Linear SVC",accuracy_score(y_test, y_pred)))
    print("Confusion Matrix:\n",metrics.confusion_matrix(y_test,y_pred))
    print("Classification Report:\n",metrics.classification_report(y_test,y_pred))
    
    # 10-fold cross validation accuracy plot
    accuracies = cross_val_score(estimator = classifier, X = x_train, y = y_train, cv = 10)
    x_range = [x for x in range(1,11)]
    ax = sns.lineplot(x=x_range,y=accuracies,markers=True,dashes=False)
    ax.set_xticks(x_range)
    ax.set_title('Linear SVC 10 fold accuracy')
    ax.set_ylabel('Acuuracy')
    ax.set_xlabel('10 Fold')
    plt.show()

#     ##################################################################################